In [ ]:
# Importing modules
import pandas as pd
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

# Loading relevant dataset
data = pd.read_csv('../data/cleaned_data.csv')

# Shifting GDP growth to compare indicators in the current year to gdp_growth the following year
data['gdp_growth_next'] = data.groupby('country')['gdp_growth'].shift(1)
data = data[data['year'] <= 2021] 

# Creating a regression input dataset
regression_data = data

In [ ]:
# Dropping observations with missing next-year GDP growth
regression_data_cleaned = regression_data.dropna(subset = ['gdp_growth_next'])

# Selecting indicators to regress next-year GDP growth on
indicators = ['debt', 'inflation', 'reserves', 'current_account', 'fdi']

# Standardising indicators (z-score normalisation)
scaler = StandardScaler()
scaled_indicators = scaler.fit_transform(regression_data_cleaned[indicators])

# Creating a standardised regression input dataset
regression_input = pd.DataFrame(scaled_indicators, columns = indicators, index = regression_data_cleaned.index)

# Reversing indicators so that a higher value always indicates higher fragility
for col in ['reserves', 'current_account', 'fdi']:
    regression_input[col] *= -1

In [ ]:
# Defining regression inputs
x = sm.add_constant(regression_input)
y = regression_data_cleaned['gdp_growth_next']

# Running the regression
regression = sm.OLS(y, x).fit()
print(regression.summary())

In [ ]:
# Extracting regression coefficients and calculating the 'fragility score'
fragility_weights = regression.params.drop('const')
data_final = regression_data_cleaned.copy()
data_final['fragility_score'] = - (regression_input @ fragility_weights)

In [ ]:
# Locate minimum and maximum fragility scores for standard min-max normalisation
fragility_minimum = data_final['fragility_score'].min()
fragility_maximum = data_final['fragility_score'].max()

# Normalise fragility scores and save the final dataset as a CSV
data_final['normalised_fragility_score'] = (data_final['fragility_score'] - fragility_minimum)/(fragility_maximum - fragility_minimum)
data_final.to_csv('../data/fragility_scores.csv', index = False)